In [3]:
from imdb import Cinemagoer, IMDbDataAccessError
import pandas as pd

In [4]:
ia = Cinemagoer()

In [5]:
# import dataset from imdb
df = pd.read_csv('./title.basics.tsv', sep='\t')
df.head()

/tmp/ipykernel_4362/1186684630.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./title.basics.tsv', sep='\t')


,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [6]:
# filter "movie"
df = df[df['titleType'] == 'movie']

In [7]:
df.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
8,tt0000009,movie,Miss Jerry,Miss Jerry,0,1894,\N,45,Romance
144,tt0000147,movie,The Corbett-Fitzsimmons Fight,The Corbett-Fitzsimmons Fight,0,1897,\N,100,"Documentary,News,Sport"
498,tt0000502,movie,Bohemios,Bohemios,0,1905,\N,100,\N
570,tt0000574,movie,The Story of the Kelly Gang,The Story of the Kelly Gang,0,1906,\N,70,"Action,Adventure,Biography"
587,tt0000591,movie,The Prodigal Son,L'enfant prodigue,0,1907,\N,90,Drama


In [8]:
# filter year between 2010 and 2024
years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']
df_movie = df[df['startYear'].isin(years)]
print(df_movie.head())

          tconst titleType                                       primaryTitle  \
11634  tt0011801     movie                                   Tötet nicht mehr   
13079  tt0013274     movie                        Istoriya grazhdanskoy voyny   
61107  tt0062336     movie  The Tango of the Widower and Its Distorting Mi...   
67659  tt0069049     movie                         The Other Side of the Wind   
69146  tt0070596     movie                                  Socialist Realism   

                                   originalTitle isAdult startYear endYear  \
11634                           Tötet nicht mehr       0      2019      \N   
13079                Istoriya grazhdanskoy voyny       0      2021      \N   
61107  El tango del viudo y su espejo deformante       0      2020      \N   
67659                 The Other Side of the Wind       0      2018      \N   
69146                     El realismo socialista       0      2023      \N   

      runtimeMinutes        genres  
11634  

In [9]:
print(df_movie.shape)
print(df_movie[df_movie['isAdult'] == 0].shape)
# df_movie['startYear'].unique()

(243822, 9)
(241197, 9)


In [10]:
# transform movie_id
df_movie['tconst'] = df_movie['tconst'].str[2:]
print(df_movie.head())

        tconst titleType                                       primaryTitle  \
11634  0011801     movie                                   Tötet nicht mehr   
13079  0013274     movie                        Istoriya grazhdanskoy voyny   
61107  0062336     movie  The Tango of the Widower and Its Distorting Mi...   
67659  0069049     movie                         The Other Side of the Wind   
69146  0070596     movie                                  Socialist Realism   

                                   originalTitle isAdult startYear endYear  \
11634                           Tötet nicht mehr       0      2019      \N   
13079                Istoriya grazhdanskoy voyny       0      2021      \N   
61107  El tango del viudo y su espejo deformante       0      2020      \N   
67659                 The Other Side of the Wind       0      2018      \N   
69146                     El realismo socialista       0      2023      \N   

      runtimeMinutes        genres  
11634             \

/tmp/ipykernel_4362/4131471320.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_movie['tconst'] = df_movie['tconst'].str[2:]


In [11]:
'''df_movie has 200k rows, which is too large for the API to handle.
Serialize the dataframe into multiple files and fetch the data concurrently by starting multiple python processes'''

import numpy as np
import pathlib
# split the dataframe into 10 files
df_movie_split = np.array_split(df_movie, 1000)
for i in range(1000):
    pathlib.Path('./work_parts').mkdir(parents=True, exist_ok=True)
    df_movie_split[i].to_csv(f'./work_parts/df_movie_{i}.csv', index=False)
